In [265]:
import os
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit


importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(setup_overlap)
importlib.reload(setup_bestfit)
importlib.reload(pipeline_lblnew)
importlib.reload(bestfit)

<module 'climatools.lblnew.bestfit_params' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/bestfit_params.py'>

In [266]:
def molbands():
    return [('h2o', '3a'),
            ('co2', '3a'),
            ('n2o', '3a')]
#    return [('h2o', '1')]
#                ('h2o', '2'),
#                ('h2o', '3a'),
#                ('h2o', '3b'),
#                ('h2o', '3c'),
#                ('h2o', '4'),
#                ('h2o', '5'),
#                ('h2o', '6'),
#                ('h2o', '7'), 
#                ('h2o', '8'),
#                ('h2o', '9'),
#                ('co2', '3a'), 
#                ('co2', '3b'),
#                ('co2', '3c'),
#                ('co2', '9'),
#                ('o3', '5'),
#                ('o3', '9'),
#                ('n2o', '3a'),
#                ('n2o', '7'),
#                ('ch4', '7') ]



def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
                     '2': '2',
                     '3a': '3',
                     '3b': '4',
                     '3c': '5',
                     '4': '6', 
                     '5': '7',
                     '6': '8',
                     '7': '9', 
                     '8': '10',
                     '9': '11'}
    
    return lblnew2clirad

In [273]:
def param_lblnew(molecule=None, band=None):
    param = bestfit.kdist_params(molecule=molecule, band=band)
    param['option_compute_ktable'] = 0
    return param


def param_cliradlw(molecule=None, band=None):
    
    param = bestfit.kdist_params(molecule=molecule, band=band)

    clirad_band = int(band_map()[band])
    conc = 'atmpro' if param['conc'] == None else param['conc']
    
    return {'atmpro': param['atmpro'],
            'band': [clirad_band],
            'commitnumber': '7e55b57',
            'molecule': {molecule: conc},
            'tsfc': param['tsfc']}


def parampairs():
    ps = []
    for molecule, band in molbands():
        for atmpro, tsfc in [('mls', 294), ('saw', 257), ('trp', 300)]:
            p_clirad = param_cliradlw(molecule=molecule, band=band)
            p_lblnew = param_lblnew(molecule=molecule, band=band)
            p_clirad['atmpro'] = atmpro
            p_clirad['tsfc'] = tsfc
            p_lblnew['atmpro'] = atmpro
            p_lblnew['tsfc'] = tsfc
            
            ps.append((p_clirad, p_lblnew))
    
    return ps


def script():
    ps = parampairs()
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps,
                                                 setup=setup)
    
    for param, _ in ps:
        print(param['molecule'], ' band', param['band'])
        print(pipeline.nbviewer_url(param=param, setup=setup))
        print()

In [274]:
script()

{'h2o': 'atmpro'}  band [3]
http://nbviewer.jupyter.org/github/qap/offline_radiation_notebooks/blob/master/longwave/lblnew_20160916/clirad/h2o_mls/band_3/atmpro_mls/cliradlw_7e55b57/results_cliradlw.ipynb

{'h2o': 'atmpro'}  band [3]
http://nbviewer.jupyter.org/github/qap/offline_radiation_notebooks/blob/master/longwave/lblnew_20160916/clirad/h2o_saw/band_3/atmpro_saw/cliradlw_7e55b57/results_cliradlw.ipynb

{'h2o': 'atmpro'}  band [3]
http://nbviewer.jupyter.org/github/qap/offline_radiation_notebooks/blob/master/longwave/lblnew_20160916/clirad/h2o_trp/band_3/atmpro_trp/cliradlw_7e55b57/results_cliradlw.ipynb

{'co2': 0.0004}  band [3]
http://nbviewer.jupyter.org/github/qap/offline_radiation_notebooks/blob/master/longwave/lblnew_20160916/clirad/co2_0.0004/band_3/atmpro_mls/cliradlw_7e55b57/results_cliradlw.ipynb

{'co2': 0.0004}  band [3]
http://nbviewer.jupyter.org/github/qap/offline_radiation_notebooks/blob/master/longwave/lblnew_20160916/clirad/co2_0.0004/band_3/atmpro_saw/cliradlw_